In [37]:
# for python 2, use print as a function
from __future__ import print_function
import pandas as pd

In [44]:
#  Representing text for the model training 
simple_train = ['call you tonight', 'call me a cab', 'please call me ... PLEASE']

In [45]:
# import and instantiate CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [46]:
# learn the vocabulary of training data
vect.fit(simple_train)

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [47]:
# examing the fitted vocabulary
vect.get_feature_names()

[u'cab', u'call', u'me', u'please', u'tonight', u'you']

In [48]:
# transofrm training data into a document term matrix
simple_train_dtm = vect.transform(simple_train)
simple_train_dtm.toarray()

array([[0, 1, 0, 0, 1, 1],
       [1, 1, 1, 0, 0, 0],
       [0, 1, 1, 2, 0, 0]])

In [49]:
pd.DataFrame(simple_train_dtm.toarray(), columns=vect.get_feature_names())

,cab,call,me,please,tonight,you
0,0,1,0,0,1,1
1,1,1,1,0,0,0
2,0,1,1,2,0,0


In [50]:
print(simple_train_dtm)

  (0, 1)	1
  (0, 4)	1
  (0, 5)	1
  (1, 0)	1
  (1, 1)	1
  (1, 2)	1
  (2, 1)	1
  (2, 2)	1
  (2, 3)	2


In [51]:
#  read file into pandas using a relative path
path = 'https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv'
sms = pd.read_table(path, header=None, names=['label', 'message'])

In [52]:
sms.shape

(5572, 2)

In [53]:
sms.head(10)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [54]:
sms.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [56]:
sms['label_num'] = sms.label.map({ 'ham': 0, 'spam': 1 })

In [57]:
sms.head(10)

,label,message,label_num
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1
6,ham,Even my brother is not like to speak with me. ...,0
7,ham,As per your request 'Melle Melle (Oru Minnamin...,0
8,spam,WINNER!! As a valued network customer you have...,1
9,spam,Had your mobile 11 months or more? U R entitle...,1


In [2]:
X = sms.message
y = sms.label_num


NameError: name 'sms' is not defined

In [1]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


NameError: name 'X' is not defined

In [61]:
# Instantiate the vectorizer
vect = CountVectorizer()

In [62]:
# Learn training data vocabulary, then use it
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)

In [63]:
# examine the document term matrix
X_train_dtm

<4179x7456 sparse matrix of type '<type 'numpy.int64'>'
	with 55209 stored elements in Compressed Sparse Row format>

In [64]:
# transform testing data into a document term
X_test_dtm = vect.transform(X_test)
X_test_dtm

<1393x7456 sparse matrix of type '<type 'numpy.int64'>'
	with 17604 stored elements in Compressed Sparse Row format>

In [65]:
# import and instantiate a multinomial naive bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [66]:
# train the model using X_train_dtm
%time nb.fit(X_train_dtm, y_train)

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 14.5 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [67]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [69]:
# calculate accuracy of class predications
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.98851399856424982

In [72]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[1203,    5],
       [  11,  174]])

In [78]:
X_test[y_pred_class > y_test]

574               Waiting for your call.
3375             Also andros ice etc etc
45      No calls..messages..missed calls
3415             No pic. Please re-send.
1988    No calls..messages..missed calls
Name: message, dtype: object